In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

import pandas as pd
from implementations import *

In [2]:
#TODO move this to a helpers folder and also why was a helper not provided anywhere?
def load_csv_data(data_path):
    """Loads data and returns y (class labels), tX (features) and ids (event ids)"""
    y = np.genfromtxt(data_path, delimiter=",", skip_header=1, dtype=str, usecols=1)
    x = np.genfromtxt(data_path, delimiter=",", skip_header=1)
    ids = x[:, 0].astype(np.int64) #check if int 64 precision was actually needed
    input_data = x[:, 2:]

    # convert class labels from strings to binary (-1,1)
    yb = np.ones(len(y))
    yb[np.where(y=='b')] = -1

    return yb, input_data, ids

In [3]:
df = pd.read_csv('train.csv')


In [4]:
print(df.columns[0])

Id


In [5]:
df.columns

Index(['Id', 'Prediction', 'DER_mass_MMC', 'DER_mass_transverse_met_lep',
       'DER_mass_vis', 'DER_pt_h', 'DER_deltaeta_jet_jet', 'DER_mass_jet_jet',
       'DER_prodeta_jet_jet', 'DER_deltar_tau_lep', 'DER_pt_tot', 'DER_sum_pt',
       'DER_pt_ratio_lep_tau', 'DER_met_phi_centrality',
       'DER_lep_eta_centrality', 'PRI_tau_pt', 'PRI_tau_eta', 'PRI_tau_phi',
       'PRI_lep_pt', 'PRI_lep_eta', 'PRI_lep_phi', 'PRI_met', 'PRI_met_phi',
       'PRI_met_sumet', 'PRI_jet_num', 'PRI_jet_leading_pt',
       'PRI_jet_leading_eta', 'PRI_jet_leading_phi', 'PRI_jet_subleading_pt',
       'PRI_jet_subleading_eta', 'PRI_jet_subleading_phi', 'PRI_jet_all_pt'],
      dtype='object')

In [6]:
df['PRI_jet_num']

0         2
1         1
2         1
3         0
4         0
         ..
249995    0
249996    0
249997    1
249998    0
249999    0
Name: PRI_jet_num, Length: 250000, dtype: int64

### Code start 

In [24]:
import datetime

DATA_TRAIN_PATH = 'train.csv'
DATA_TEST_PATH = 'test.csv'
y_train, tX_train, ids_train = load_csv_data(DATA_TRAIN_PATH)
y_test, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)


In [25]:
def clean_data(data):
    data_cleaned = data
    
    data_cleaned[data_cleaned == -999] = np.NaN
    
    #replace NaN's by mean of columns
    medians = np.nanmedian(data_cleaned, axis=0)
    sq_std = np.std(data_cleaned, axis=0) **2
    inds = np.where(np.isnan(data_cleaned))
    data_cleaned[inds] = np.take(medians, inds[1])
    
    #standardize the columns 
    data_cleaned = (data_cleaned - medians)  / sq_std 
    
    
    #augment the data 
    
    
    
    return data 

In [26]:
def split_data(x, y, ratio, seed=1):
    """split the dataset based on the split ratio."""
    # set seed
    np.random.seed(seed)
    # generate random indices
    num_row = len(y)
    indices = np.random.permutation(num_row)
    index_split = int(np.floor(ratio * num_row))
    index_tr = indices[: index_split]
    index_te = indices[index_split:]
    # create split
    x_tr = x[index_tr]
    x_te = x[index_te]
    y_tr = y[index_tr]
    y_te = y[index_te]
    return x_tr, x_te, y_tr, y_te

In [45]:
def real_positives(pred, act):
    tot = 0
    good = 0
    for p,a in zip(pred,act):
        if(a == 1.0):
            tot += 1
            if(p == 1.0):
                good += 1
    return good / tot
                
            
def real_negatives(pred,act):
    tot = 0
    good = 0
    for p,a in zip(pred,act):
        if(a == 0.0):
            tot += 1
            if(p == 0.0):
                good += 1
    return good/tot
        

In [27]:
#cleaning the training and test set
tX_train_clean = clean_data(tX_train)
tX_test_clean = clean_data(tX_test)

(30,)
(30,)


In [30]:
#splitting the dataset into train and validation 
xTr, xVal, yTr, yVal = split_data(tX_train_clean, y_train, ratio=0.75, seed= 1)

In [31]:
#train the model on the training set 

In [50]:
w, loss = ridge_regression(yTr, xTr, 0)
print(loss)

0.3438575819298831


In [51]:
#predict on the validation set 
pred = xVal.dot(w)

#format the predictions  
pred[pred > 0] = 1
pred[pred < 0] = -1 

#precision 
errors = np.sum(np.abs((yVal - pred)))


#real_positives = real_positives(pred, yVal) 
#real_negatives = real_negatives(pred, yVal)
#TODO: add the F1 score calculation as well
print("the precision is :")
print(errors / len(yVal))
print("the real positives are :")
#print(real_positives)
print("the real negatives are :")
#print(real_negatives)


the precision is :
0.519392
the real positives are :
the real negatives are :


In [92]:
import csv 
def create_csv_submission(ids, y_pred, name):
    """
    Creates an output file in .csv format for submission to Kaggle or AIcrowd
    Arguments: ids (event ids associated with each prediction)
               y_pred (predicted class labels)
               name (string name of .csv output file to be created)
    """
    with open(name, "w") as csvfile:
        fieldnames = ["Id", "Prediction"]
        writer = csv.DictWriter(csvfile, delimiter=",", fieldnames=fieldnames)
        writer.writeheader()
        for r1, r2 in zip(ids, y_pred):
            writer.writerow({"Id": int(r1), "Prediction": int(r2)})


In [93]:
create_csv_submission(ids_test, pred, "pred1.csv")